<a href="https://colab.research.google.com/github/xtribr/nlpenem/blob/main/Ollamadeepsekenem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- INSTALAÇÃO MESTRA (SEM ERROS) ---
import os
import subprocess

print("☢️  Iniciando Limpeza e Instalação Blindada...")

# 1. Desinstalar forçadamente versões conflituosas
pkgs = ["faiss-gpu", "faiss-cpu", "langchain", "langchain-community", "transformers", "accelerate", "bitsandbytes"]
for pkg in pkgs:
    subprocess.run(["pip", "uninstall", "-y", pkg], check=False)

print("⬇️  Instalando Pacote Sênior...")

# 2. Comandos de instalação silenciosa (-q) e atualização (-U)
comandos = [
    # Base do Modelo (GPU)
    "pip install -q -U torch torchvision torchaudio",
    "pip install -q -U bitsandbytes",
    "pip install -q -U git+https://github.com/huggingface/transformers.git",
    "pip install -q -U git+https://github.com/huggingface/accelerate.git",

    # Base do RAG (Dados/Memória) - Usando CPU para evitar erro de driver
    "pip install -q -U langchain langchain-community langchain-core",
    "pip install -q -U faiss-cpu",
    "pip install -q -U sentence-transformers",
    "pip install -q -U gdown"
]

for cmd in comandos:
    print(f"   Executando: {cmd.split()[2]}...") # Mostra o nome do pacote
    os.system(cmd)

print("\n✅ AMBIENTE PRONTO! \n⚠️  IMPORTANTE: Vá em 'Ambiente de Execução' > 'Reiniciar Sessão' antes de continuar.")

☢️  Iniciando Limpeza e Instalação Blindada...
⬇️  Instalando Pacote Sênior...
   Executando: -q...
   Executando: -q...
   Executando: -q...
   Executando: -q...
   Executando: -q...
   Executando: -q...
   Executando: -q...
   Executando: -q...

✅ AMBIENTE PRONTO! 
⚠️  IMPORTANTE: Vá em 'Ambiente de Execução' > 'Reiniciar Sessão' antes de continuar.


In [1]:
# 1. Removemos tentativas falhas anteriores
!pip uninstall -y faiss-gpu

# 2. Instalamos o faiss-cpu (Universal) + LangChain atualizado
!pip install -q -U langchain langchain-community langchain-core sentence-transformers faiss-cpu

print("✅ Dependências instaladas (Versão CPU estável)!")


✅ Dependências instaladas (Versão CPU estável)!


In [14]:
import torch
import os
from google.colab import drive
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Montar Drive
drive.mount('/content/drive')

# Verificar se o arquivo está lá mesmo
MODEL_PATH = "/content/drive/MyDrive/modelos/sabia-7b"
if os.path.exists(MODEL_PATH):
    print(f"✅ Caminho confirmado: {MODEL_PATH}")
else:
    print(f"❌ ERRO: A pasta {MODEL_PATH} não foi encontrada. Verifique seu Drive.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Caminho confirmado: /content/drive/MyDrive/modelos/sabia-7b


In [15]:
# Configuração de Quantização 4-bit (NF4)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

print("⏳ Carregando modelo na GPU (Isso leva ~1 a 2 minutos)...")

try:
    # 1. Carregar Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)

    # 2. Carregar Modelo com mapeamento FORÇADO para GPU 0
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        quantization_config=bnb_config,
        device_map={"": 0},  # <--- O SEGREDO: Força tudo para a GPU 0
        local_files_only=True,
        trust_remote_code=True # Necessário para alguns modelos da família Llama/Sabiá
    )

    print("\n✅ SUCESSO! O Sabiá-7B está carregado e pronto.")

except Exception as e:
    print("\n❌ ERRO FATAL:")
    print(e)

⏳ Carregando modelo na GPU (Isso leva ~1 a 2 minutos)...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]


✅ SUCESSO! O Sabiá-7B está carregado e pronto.


In [16]:
prompt_usuario = """Você é um especialista em exames.
Questão do ENEM:
"A falta de saneamento básico e o descarte irregular de lixo favorecem a proliferação do mosquito Aedes aegypti."
Explique qual doença é transmitida por este vetor e como preveni-la.

Resposta:"""

# Preparar entrada
inputs = tokenizer(prompt_usuario, return_tensors="pt").to("cuda")

print("🤖 Gerando resposta...")

# Gerar
outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    temperature=0.7,
    repetition_penalty=1.2,
    do_sample=True
)

# Mostrar resultado
texto_final = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("-" * 30)
print(texto_final)
print("-" * 30)

🤖 Gerando resposta...
------------------------------
Você é um especialista em exames.
Questão do ENEM:
"A falta de saneamento básico e o descarte irregular de lixo favorecem a proliferação do mosquito Aedes aegypti."
Explique qual doença é transmitida por este vetor e como preveni-la.

Resposta:
Ovos de moscas do gênero Aedes, da família Culicidae, depositados na água limpa, são eclodidos e as ninfas, semelhantes ao adulto, se alimentam de sangue de animais vertebrados e invertebrados e se desenvolvem na água. Após a alimentação, as ninfas voltam para a água e se transformam em adultos. Esses adultos se alimentam de polpa e néctar e se reproduzem, liberando ovos em água limpa.





































































------------------------------


In [17]:
# Instala biblioteca de download do Drive
!pip install -q gdown

# Baixa a pasta inteira direto do link que você passou
# O ID '1datullhe8eo6Ogi5zVV04TJyRl314eDZ' é o código da pasta no seu link
print("⬇️ Baixando dataset do ENEM...")
!gdown --folder 1datullhe8eo6Ogi5zVV04TJyRl314eDZ -O enem_dados --quiet

print("✅ Download concluído! Verificando arquivos:")
import os
if os.path.exists("enem_dados"):
    arquivos = os.listdir("enem_dados")
    print(f"📂 Encontrados {len(arquivos)} arquivos na pasta 'enem_dados'.")
    print(f"   Exemplo: {arquivos[:3]}") # Mostra os 3 primeiros nomes
else:
    print("❌ Erro: Pasta não criada.")

⬇️ Baixando dataset do ENEM...
✅ Download concluído! Verificando arquivos:
📂 Encontrados 21 arquivos na pasta 'enem_dados'.
   Exemplo: ['enem_2012_completo.jsonl', 'enem_2023_completo.jsonl', 'enem_2020_completo.jsonl']


In [18]:
import json
import glob
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Padrão para pegar os arquivos que você baixou
JSONL_PATTERN = "enem_dados/*.jsonl"

def carregar_jsonl_enem():
    arquivos = glob.glob(JSONL_PATTERN)
    docs_para_indexar = []
    total_questoes = 0

    print(f"📚 Iniciando leitura de {len(arquivos)} arquivos .jsonl...")

    for arquivo in arquivos:
        try:
            with open(arquivo, 'r', encoding='utf-8') as f:
                for linha in f:
                    if not linha.strip(): continue # Pula linhas vazias

                    # Lê o objeto JSON da linha
                    item = json.loads(linha)

                    # --- EXTRAÇÃO INTELIGENTE DE DADOS ---
                    # Adaptação para campos comuns em datasets de NLP do ENEM
                    texto_base = item.get('question') or item.get('enunciado') or item.get('text') or ""

                    # Tratamento das alternativas (se for lista ou dict)
                    alts = item.get('alternatives') or item.get('alternativas') or []
                    texto_alternativas = ""
                    if isinstance(alts, list):
                        letras = ['A', 'B', 'C', 'D', 'E']
                        for i, alt in enumerate(alts):
                            if i < 5: texto_alternativas += f"{letras[i]}) {alt}\n"
                    elif isinstance(alts, dict):
                        for k, v in alts.items(): texto_alternativas += f"{k}) {v}\n"

                    gabarito = item.get('correct_answer') or item.get('gabarito') or item.get('label')
                    ano = item.get('year') or item.get('ano') or "S/D"

                    # Se tiver conteúdo suficiente, cria o "Cartão de Memória"
                    if texto_base:
                        conteudo = f"""
[QUESTÃO ENEM {ano}]
ENUNCIADO: {texto_base}
ALTERNATIVAS:
{texto_alternativas}
GABARITO OFICIAL: {gabarito}
"""
                        docs_para_indexar.append(conteudo)
                        total_questoes += 1

        except Exception as e:
            print(f"⚠️ Erro ao processar arquivo {arquivo}: {e}")

    print(f"✅ Processamento finalizado! {total_questoes} questões indexadas.")
    return docs_para_indexar

# --- EXECUÇÃO DO INDEXADOR ---
textos = carregar_jsonl_enem()

if textos:
    print("🧠 Gerando Embeddings e Criando Banco Vetorial (Isso leva uns 2-3 min)...")
    # Modelo Multilingue Rápido
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

    # Cria o banco FAISS
    vector_db = FAISS.from_texts(textos, embeddings)
    print("🏆 Banco de Dados pronto! O Sabiá agora tem memória de todas essas provas.")
else:
    print("❌ Erro: Nenhuma questão encontrada. Verifique se os arquivos .jsonl estão na pasta correta.")

📚 Iniciando leitura de 21 arquivos .jsonl...
✅ Processamento finalizado! 3084 questões indexadas.
🧠 Gerando Embeddings e Criando Banco Vetorial (Isso leva uns 2-3 min)...
🏆 Banco de Dados pronto! O Sabiá agora tem memória de todas essas provas.


In [19]:
import json
import glob
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Padrão para pegar os arquivos que você baixou
JSONL_PATTERN = "enem_dados/*.jsonl"

def carregar_jsonl_enem():
    arquivos = glob.glob(JSONL_PATTERN)
    docs_para_indexar = []
    total_questoes = 0

    print(f"📚 Lendo {len(arquivos)} arquivos de provas (.jsonl)...")

    for arquivo in arquivos:
        try:
            with open(arquivo, 'r', encoding='utf-8') as f:
                for linha in f:
                    if not linha.strip(): continue # Pula linhas em branco

                    # Lê cada questão individualmente
                    item = json.loads(linha)

                    # --- EXTRAÇÃO DE DADOS ---
                    # Tenta capturar o texto da questão e o gabarito
                    texto_base = item.get('question') or item.get('enunciado') or item.get('text') or item.get('body') or ""
                    gabarito = item.get('correct_answer') or item.get('gabarito') or item.get('label')
                    ano = item.get('year') or item.get('ano') or "S/D"

                    # Formata as alternativas se existirem
                    alts = item.get('alternatives') or item.get('alternativas') or []
                    texto_alts = ""
                    if isinstance(alts, list):
                        letras = ['A', 'B', 'C', 'D', 'E']
                        for i, a in enumerate(alts):
                            if i < 5: texto_alts += f"{letras[i]}) {a}\n"
                    elif isinstance(alts, dict):
                         for k, v in alts.items(): texto_alts += f"{k}) {v}\n"

                    # Só indexa se tiver texto
                    if texto_base and len(texto_base) > 10:
                        conteudo = f"""
[PROVA ENEM {ano}]
ENUNCIADO: {texto_base}
ALTERNATIVAS:
{texto_alts}
GABARITO: {gabarito}
"""
                        docs_para_indexar.append(conteudo)
                        total_questoes += 1

        except Exception as e:
            pass # Segue o jogo se um arquivo der erro

    print(f"✅ Sucesso! {total_questoes} questões foram preparadas para a memória.")
    return docs_para_indexar

# --- CRIAÇÃO DO BANCO DE DADOS ---
textos = carregar_jsonl_enem()

if textos:
    print("🧠 Criando a 'Memória Neural' (Indexando vetores)...")
    # Usa embeddings leves para ser rápido
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

    # Cria o índice FAISS
    vector_db = FAISS.from_texts(textos, embeddings)
    print("🏆 Banco de Dados pronto! O Sabiá agora conhece todas essas questões.")
else:
    print("❌ Erro: Não encontrei questões. Verifique se a pasta 'enem_dados' existe.")

📚 Lendo 21 arquivos de provas (.jsonl)...
✅ Sucesso! 3082 questões foram preparadas para a memória.
🧠 Criando a 'Memória Neural' (Indexando vetores)...
🏆 Banco de Dados pronto! O Sabiá agora conhece todas essas questões.


In [20]:
import torch

# --- CONFIGURAÇÃO DE SEGURANÇA ---
# Garante que o modelo saiba o que é um "fim de frase"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def professor_enem(pergunta):
    if 'vector_db' not in globals():
        print("❌ Banco de dados não carregado.")
        return

    print(f"\n🔎 RAG: Buscando contexto para '{pergunta}'...")

    # 1. Busca contexto
    docs = vector_db.similarity_search(pergunta, k=3)
    contexto_str = "\n\n".join([f"FONTE {i+1}: {d.page_content}" for i, d in enumerate(docs)])

    # 2. Prompt Estruturado (Formato Alpaca/Llama que funciona bem no Sabiá)
    prompt_sistema = f"""Abaixo está uma instrução que descreve uma tarefa, acompanhada de um contexto que fornece mais informações. Escreva uma resposta que complete a solicitação adequadamente.

### Instrução:
Você é um Professor do ENEM. Responda à dúvida do aluno com base APENAS no contexto fornecido. Se o contexto não tiver a resposta, diga que não sabe.

### Contexto:
{contexto_str}

### Dúvida do Aluno:
{pergunta}

### Resposta:"""

    # 3. Tokenização Manual (Mais segura)
    inputs = tokenizer(prompt_sistema, return_tensors="pt").to("cuda")

    print("🤖 Sabiá pensando...")

    # 4. Geração com Parâmetros Anti-Loop
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=300,      # Limite de tamanho
            temperature=0.2,         # Baixa temperatura = menos alucinação
            top_p=0.9,
            repetition_penalty=1.2,  # FORÇA a não repetir (mata o bug do espaço em branco)
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    # 5. Decodificação Limpa
    # Pega apenas os tokens novos gerados (corta o prompt fora)
    texto_gerado = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    print("\n" + "="*50)
    if not texto_gerado.strip():
        print("⚠️ O modelo gerou vazio. Tentando prompt alternativo...")
        # Fallback simples se falhar
    else:
        print(texto_gerado.strip())
    print("="*50)

# --- TESTE ---
professor_enem("O que é variação linguística e como ela cai na prova?")


🔎 RAG: Buscando contexto para 'O que é variação linguística e como ela cai na prova?'...
🤖 Sabiá pensando...

A variação linguística é um fenômeno natural da língua, que ocorre em função de fatores sociais, culturais, históricos, regionais, entre outros.

A variação linguística pode ser observada em diferentes níveis:

- Variação diafásica: ocorre em função da situação comunicativa, ou seja, da situação de comunicação.

- Variação situacional: ocorre em função da situação comunicativa, ou seja, da situação de comunicação.

- Variação diastrática: ocorre em função da situação comunicativa, ou seja, da situação de comunicação.

- Variação geográfica: ocorre em função da região, ou seja, do local onde o falante vive.

- Variação histórica: ocorre em função do tempo, ou seja, do momento histórico em que o falante vive.

- Variação social: ocorre em função da posição social do falante, ou seja, da posição social do falante.

- Variação estilística: ocorre em função da intenção do falante, 

In [14]:
# Célula de Instalação Segura
!pip install -q pdfplumber
print("✅ Biblioteca pdfplumber instalada com sucesso.")

✅ Biblioteca pdfplumber instalada com sucesso.


In [18]:
# 1. Instalação específica do módulo que está faltando
!pip install -q langchain-text-splitters pdfplumber

# 2. Código de Ingestão (com o import atualizado e blindado)
import os
import pdfplumber

# Tentativa de importação robusta (tenta o novo, se falhar tenta o antigo)
try:
    from langchain_text_splitters import RecursiveCharacterTextSplitter
except ImportError:
    from langchain.text_splitter import RecursiveCharacterTextSplitter

# Seus arquivos (Verifique se estão na aba lateral)
PAPERS_FILES = ["sabia-enem.pdf", "2304.07880v4.pdf"]

def injetar_inteligencia_tecnica():
    print("🔬 Lendo artigos científicos (Papers Maritaca AI)...")

    texto_acumulado = ""
    arquivos_processados = 0

    for pdf_file in PAPERS_FILES:
        if os.path.exists(pdf_file):
            print(f"   📖 Extraindo texto de: {pdf_file}...")
            try:
                with pdfplumber.open(pdf_file) as pdf:
                    for page in pdf.pages:
                        t = page.extract_text()
                        if t: texto_acumulado += t + "\n"
                arquivos_processados += 1
            except Exception as e:
                print(f"   ⚠️ Erro ao ler {pdf_file}: {e}")
        else:
            print(f"   ⚠️ Arquivo não encontrado: {pdf_file} (Verifique o upload)")

    if arquivos_processados == 0:
        print("❌ Nenhum arquivo lido. Abortando ingestão.")
        return

    # Dividir o texto usando a metodologia do paper (Chunks maiores para contexto)
    print(f"   Processando conhecimento técnico...")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=150,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    docs_tecnicos = text_splitter.split_text(texto_acumulado)

    # Inserir no Banco de Dados
    if 'vector_db' in globals():
        vector_db.add_texts(docs_tecnicos)
        print(f"\n✅ SUCESSO! {len(docs_tecnicos)} blocos de 'Metodologia Sabiá' foram adicionados à memória.")
        print("   O modelo agora tem acesso às técnicas de CoT (Chain of Thought) descritas nos papers.")
    else:
        print("❌ Erro: O 'vector_db' não existe. Você precisa rodar a Célula 7 antes.")

# Executar
injetar_inteligencia_tecnica()

🔬 Lendo artigos científicos (Papers Maritaca AI)...
   📖 Extraindo texto de: sabia-enem.pdf...
   📖 Extraindo texto de: 2304.07880v4.pdf...
   Processando conhecimento técnico...

✅ SUCESSO! 99 blocos de 'Metodologia Sabiá' foram adicionados à memória.
   O modelo agora tem acesso às técnicas de CoT (Chain of Thought) descritas nos papers.


In [21]:
import torch

def professor_enem_pragmatico(pergunta):
    if 'vector_db' not in globals():
        print("❌ Banco de dados off. Rode a Célula 7.")
        return

    print(f"📝 Consultando banco do ENEM para resolver: '{pergunta[:50]}...'")

    # 1. Busca questões parecidas no histórico (Sua base JSONL)
    docs = vector_db.similarity_search(pergunta, k=3)

    # Monta os exemplos ("Colas")
    # Limita o tamanho de cada exemplo para não estourar a memória
    exemplos = ""
    for i, d in enumerate(docs):
        exemplos += f"EXEMPLO {i+1}:\n{d.page_content[:400]}...\n\n"

    # 2. Prompt "Curto e Grosso" (Funciona melhor em 4-bit)
    prompt = f"""Você é um Professor do ENEM. Use os exemplos abaixo para entender o padrão e resolva a QUESTÃO ATUAL passo a passo.

BASE DE CONHECIMENTO (Questões Anteriores):
{exemplos}
===================================

QUESTÃO ATUAL:
{pergunta}

RESOLUÇÃO PASSO A PASSO (Chain-of-Thought):
1. Análise do Enunciado:"""

    # 3. Tokenização
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # 4. Geração (Com empurrãozinho de temperatura)
    print("🤖 Sabiá escrevendo resolução...")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=400,
            do_sample=True,        # Ligamos o sample para evitar o bug do vazio
            temperature=0.1,       # Baixíssima temperatura (quase determinístico)
            top_p=0.95,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id
        )

    # 5. Limpeza
    # Cortamos o prompt e adicionamos o início que forçamos ("1. Análise...")
    texto_gerado = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    print("\n" + "█"*60)
    print("1. Análise do Enunciado:" + texto_gerado)
    print("█"*60)

# --- TESTE REAL COM QUESTÃO TÍPICA ENEM ---
# Questão sobre Ciclo do Carbono (Biologia/Química)
questao_enem = """
(ENEM) O uso de combustíveis fósseis interfere no ciclo do carbono, pois provoca o aumento da quantidade de carbono na atmosfera.
A queima desses combustíveis libera gases que intensificam qual fenômeno ambiental?
A) Chuva ácida
B) Efeito estufa
C) Destruição da camada de ozônio
D) Eutrofização das águas
E) Inversão térmica
"""

professor_enem_pragmatico(questao_enem)

📝 Consultando banco do ENEM para resolver: '
(ENEM) O uso de combustíveis fósseis interfere no...'
🤖 Sabiá escrevendo resolução...

████████████████████████████████████████████████████████████
1. Análise do Enunciado:

O uso de combustíveis fósseis interfere no ciclo do carbono, pois provoca o aumento da quantidade de carbono na atmosfera.

2. Análise das Alternativas:

A) Chuva ácida

B) Efeito estufa

C) Destruição da camada de ozônio

D) Eutrofização das águas

E) Inversão térmica

































































































































































































































































































████████████████████████████████████████████████████████████


In [22]:
import random
import re
import torch

def simulado_local_definitivo(qtd_questoes=5):
    # Verificação básica
    if 'model' not in globals() or 'tokenizer' not in globals():
        print("❌ ERRO: O modelo Sabiá-7B não está carregado na memória. Rode a Célula 4.")
        return
    if 'textos' not in globals() or not textos:
        print("❌ ERRO: As provas não foram carregadas. Rode a Célula 7.")
        return

    print(f"🔥 INICIANDO SIMULADO LOCAL (Sabiá-7B 4-bit) - {qtd_questoes} Questões")
    print("Aplicando metodologia CoT (Chain-of-Thought) descrita no paper...")

    amostra = random.sample(textos, qtd_questoes)
    acertos = 0
    total = 0

    for i, questao_raw in enumerate(amostra):
        try:
            # 1. Preparação dos Dados
            partes = questao_raw.split("GABARITO:")
            enunciado_limpo = partes[0].replace("[QUESTÃO ENEM", "").strip()
            # Removemos cabeçalhos soltos para limpar a visão do modelo
            enunciado_limpo = enunciado_limpo.split("ENUNCIADO:")[-1].strip()

            gabarito_real = partes[1].strip()[0].upper()

            print(f"\n🔸 Questão {i+1}: Gabarito Oficial [{gabarito_real}]")

            # 2. RAG: Buscar contexto (Questões parecidas para servir de exemplo)
            docs = vector_db.similarity_search(enunciado_limpo, k=1) # 1 exemplo basta para few-shot
            exemplo_rag = docs[0].page_content

            # Tenta extrair o gabarito do exemplo do RAG para ensinar o modelo
            gabarito_exemplo = "A" # Fallback
            if "GABARITO:" in exemplo_rag:
                gabarito_exemplo = exemplo_rag.split("GABARITO:")[1].strip()[0]

            # 3. PROMPT ENGENHARIA REVERSA (Baseado no Paper Sabiá)
            # O paper sugere dar exemplos resolvidos. Vamos simular isso.
            prompt = f"""Você é um assistente especialista no ENEM. Siga o raciocínio abaixo.

EXEMPLO DE RESOLUÇÃO:
Questão: {exemplo_rag.split('GABARITO:')[0].strip()}
Raciocínio: Analisando o enunciado e as alternativas, a resposta correta é a alternativa {gabarito_exemplo}.
Resposta: {gabarito_exemplo}

AGORA É SUA VEZ:
Questão: {enunciado_limpo}
Raciocínio:"""

            # 4. GERAÇÃO DETERMINÍSTICA (Greedy Decoding)
            # O paper recomenda temperatura 0 (greedy) para provas.
            inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=200,    # Curto para ele não divagar
                    do_sample=False,       # Greedy Search (Determinístico - vital para 4-bit)
                    repetition_penalty=1.2,# Penaliza repetição fortemente
                    pad_token_id=tokenizer.eos_token_id,
                    eos_token_id=tokenizer.eos_token_id
                )

            # 5. Extração
            texto_gerado = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

            # Limpeza bruta: Pega a primeira letra que aparecer depois de "Resposta:"
            # Ou tenta achar qualquer menção a letra isolada
            predicao = "?"

            # Estratégia 1: Procura explícita "Resposta: X"
            match = re.search(r'Resposta:.*?([A-E])', texto_gerado, re.IGNORECASE | re.DOTALL)
            if match:
                predicao = match.group(1).upper()
            else:
                # Estratégia 2: Procura "Alternativa X"
                match_alt = re.search(r'Alternativa.*?([A-E])', texto_gerado, re.IGNORECASE)
                if match_alt:
                    predicao = match_alt.group(1).upper()
                else:
                    # Estratégia 3: A última letra maiúscula isolada do texto
                    # Isso pega casos onde ele diz "Logo, é a B."
                    match_last = re.findall(r'\b([A-E])\b', texto_gerado)
                    if match_last: predicao = match_last[-1]

            print(f"   🤖 Sabiá Local: {predicao}")
            # print(f"   (Debug Texto: {texto_gerado.strip()[:100]}...)") # Descomente se der erro

            if predicao == gabarito_real:
                print("   ✅ ACERTOU")
                acertos += 1
            else:
                print("   ❌ ERROU")

            total += 1

        except Exception as e:
            print(f"   ⚠️ Erro de execução: {e}")
            continue

    # --- PLACAR ---
    if total > 0:
        nota = (acertos / total) * 100
        print("\n" + "█"*50)
        print(f"📊 RESULTADO LOCAL (Sabiá-7B): {nota}%")
        print("█"*50)

# Executar Simulado
simulado_local_definitivo(10)

🔥 INICIANDO SIMULADO LOCAL (Sabiá-7B 4-bit) - 10 Questões
Aplicando metodologia CoT (Chain-of-Thought) descrita no paper...

🔸 Questão 1: Gabarito Oficial [D]
   🤖 Sabiá Local: D
   ✅ ACERTOU

🔸 Questão 2: Gabarito Oficial [C]
   🤖 Sabiá Local: C
   ✅ ACERTOU

🔸 Questão 3: Gabarito Oficial [A]
   🤖 Sabiá Local: A
   ✅ ACERTOU

🔸 Questão 4: Gabarito Oficial [C]
   🤖 Sabiá Local: C
   ✅ ACERTOU

🔸 Questão 5: Gabarito Oficial [E]
   🤖 Sabiá Local: E
   ✅ ACERTOU

🔸 Questão 6: Gabarito Oficial [B]
   🤖 Sabiá Local: B
   ✅ ACERTOU

🔸 Questão 7: Gabarito Oficial [C]
   🤖 Sabiá Local: C
   ✅ ACERTOU

🔸 Questão 8: Gabarito Oficial [B]
   🤖 Sabiá Local: B
   ✅ ACERTOU

🔸 Questão 9: Gabarito Oficial [C]
   🤖 Sabiá Local: C
   ✅ ACERTOU

🔸 Questão 10: Gabarito Oficial [D]
   🤖 Sabiá Local: D
   ✅ ACERTOU

██████████████████████████████████████████████████
📊 RESULTADO LOCAL (Sabiá-7B): 100.0%
██████████████████████████████████████████████████


In [23]:
import requests
import json
import random
import re
import time

# --- 1. CONFIGURAÇÃO DA API ---
# Cole sua chave aqui
API_KEY = "107341642936117619902_14127420ffa6b338"
ENDPOINT = "https://chat.maritaca.ai/api/chat/completions"

# --- 2. SEU PROMPT MESTRE (CÓPIA EXATA) ---
# Aqui está o texto integral que você pediu, sem resumos.
METODOLOGIA_EXTREMA = """
VOCÊ É O SABIÁ-3.1. SIGA ESTE PROTOCOLO RIGOROSO PARA RESOLVER A QUESTÃO:

📋 METODOLOGIA DETALHADA:

PASSO 1: ANÁLISE INICIAL PROFUNDA
- Leia o contexto COMPLETO com máxima atenção
- Identifique TODOS os dados fornecidos (explícitos e implícitos)
- Identifique o que está sendo pedido (pode haver múltiplas etapas)
- Identifique o tipo de problema matemático
- Anote unidades de medida e relações entre dados

PASSO 2: PLANEJAMENTO ESTRATÉGICO
- Determine qual(is) conceito(s) matemático(s) aplicar
- Identifique se há múltiplas etapas na resolução
- Planeje TODOS os passos antes de começar
- Identifique fórmulas necessárias
- Verifique se há conversões de unidades necessárias
- Identifique possíveis armadilhas ou pegadinhas

PASSO 3: RESOLUÇÃO PASSO A PASSO DETALHADA
- Resolva o problema passo a passo
- Mostre TODOS os cálculos intermediários
- Verifique cada operação matemática
- Mantenha precisão numérica (cuidado com arredondamentos)
- Se usar aproximações, anote claramente
- Se houver múltiplas etapas, valide cada uma antes de prosseguir

PASSO 4: VALIDAÇÃO MÚLTIPLA
- Valide usando método inverso (substituir na equação original)
- Verifique se a resposta faz sentido no contexto
- Verifique se a resposta está nas unidades corretas
- Verifique se a resposta responde à pergunta feita
- Verifique se não há erros de cálculo ou interpretação

PASSO 5: ANÁLISE DETALHADA DE CADA ALTERNATIVA
Para CADA alternativa (A, B, C, D, E):
- Calcule o valor numérico (se aplicável)
- Compare com sua resposta calculada
- Identifique se há erros comuns que levariam a essa alternativa:
  * Erros de cálculo
  * Erros de interpretação
  * Erros de conversão de unidades
  * Erros de aplicação de fórmulas
  * Erros de sinal ou operação
- Elimine alternativas claramente incorretas
- Justifique por que cada alternativa está correta ou incorreta

PASSO 6: ELIMINAÇÃO E ESCOLHA FINAL
- Elimine alternativas que você identificou como incorretas
- Entre as alternativas restantes, compare cuidadosamente
- Se houver dúvida entre duas alternativas:
  * Refaça os cálculos críticos
  * Verifique se não houve erro de sinal ou operação
  * Valide com método inverso
  * Foque na diferença entre as alternativas
- Escolha a alternativa que corresponde EXATAMENTE à sua resposta calculada

PASSO 7: VERIFICAÇÃO FINAL RIGOROSA
Antes de responder, confirme:
- ✅ Minha resposta calculada corresponde a qual alternativa?
- ✅ Eliminei as alternativas incorretas?
- ✅ Validei com método inverso?
- ✅ Verifiquei unidades e contexto?
- ✅ Verifiquei se não há erros de cálculo?
- ✅ Verifiquei se não há erros de interpretação?
- ✅ A resposta faz sentido matematicamente e contextualmente?

🎯 INSTRUÇÕES ESPECÍFICAS PARA QUESTÕES DIFÍCEIS:

1. PRECISÃO NUMÉRICA MÁXIMA:
   - Mantenha casas decimais adequadas durante os cálculos
   - Cuidado com arredondamentos prematuros
   - Use frações quando possível para maior precisão
   - Valide resultados aproximados

2. INTERPRETAÇÃO CUIDADOSA:
   - Leia cuidadosamente eixos e legendas (se houver gráfico)
   - Identifique escalas e unidades
   - Extraia dados corretamente
   - Cuidado com interpretações literais vs. matemáticas

3. PROBLEMAS CONTEXTUALIZADOS:
   - Relacione o problema matemático com o contexto real
   - Verifique se sua resposta faz sentido prático
   - Cuidado com interpretações literais vs. matemáticas

4. MÚLTIPLAS ETAPAS:
   - Resolva cada etapa separadamente
   - Valide cada etapa antes de prosseguir
   - Verifique se todas as etapas foram completadas

5. ELIMINAÇÃO DE ALTERNATIVAS:
   - Use estimativas para eliminar alternativas absurdas
   - Compare ordens de grandeza
   - Verifique se alternativas estão em unidades corretas

⚠️ LEMBRE-SE:
- Questões difíceis exigem cuidado extra
- SEMPRE valide antes de escolher
- NÃO escolha por "intuição" - use cálculo e validação
- Se estiver em dúvida, refaça os cálculos focando na diferença entre alternativas

⚠️ INSTRUÇÃO FINAL OBRIGATÓRIA:
Após seguir rigorosamente todos os passos acima, pule uma linha e escreva EXATAMENTE:
RESPOSTA FINAL: [Letra]
"""

# --- 3. FUNÇÃO DE CHAMADA API ---
def chamar_maritaca_full(pergunta, contexto):
    if "INSIRA" in API_KEY: return "ERRO: Configure a API Key na linha 9."

    # Junta tudo: Metodologia Completa + Contexto RAG + Pergunta
    prompt_sistema = f"""{METODOLOGIA_EXTREMA}

📚 CONTEXTO DE APOIO (Questões Anteriores):
{contexto}

❓ QUESTÃO PARA RESOLVER AGORA:
{pergunta}
"""

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "sabia-3.1",
        "messages": [
            {"role": "user", "content": prompt_sistema}
        ],
        "temperature": 0.0, # Zero criatividade para seguir o protocolo à risca
        "max_tokens": 2500  # Aumentei para 2500 pois o prompt de saída será longo
    }

    try:
        response = requests.post(ENDPOINT, headers=headers, json=payload)

        if response.status_code == 200:
            return response.json()['choices'][0]['message']['content']
        else:
            return f"ERRO API {response.status_code}: {response.text}"

    except Exception as e:
        return f"ERRO REQUISIÇÃO: {str(e)}"

# --- 4. EXECUTAR SIMULADO ---
def rodar_simulado_full(qtd=3):
    if 'textos' not in globals():
        print("❌ ERRO: Rode a Célula 7 (Carregar Dados) primeiro.")
        return

    print(f"🔥 INICIANDO SIMULADO COM METODOLOGIA COMPLETA (Sabiá-3.1)...")

    amostra = random.sample(textos, qtd)
    acertos = 0
    total = 0

    for i, item in enumerate(amostra):
        try:
            partes = item.split("GABARITO:")
            if len(partes) < 2: continue

            enunciado = partes[0].replace("[QUESTÃO ENEM", "").split("ENUNCIADO:")[-1].strip()
            gabarito_oficial = partes[1].strip()[0].upper()

            print(f"\n{'='*60}")
            print(f"🔸 QUESTÃO {i+1} | Gabarito Oficial: [{gabarito_oficial}]")

            # Contexto RAG
            contexto = ""
            if 'vector_db' in globals():
                docs = vector_db.similarity_search(enunciado, k=3)
                contexto = "\n".join([d.page_content[:300] for d in docs])

            print("🤔 Sabiá-3.1 aplicando os 7 passos (Pode demorar +20s)...")
            inicio = time.time()
            resposta_completa = chamar_maritaca_full(enunciado, contexto)
            tempo = time.time() - inicio

            if "ERRO" in resposta_completa[:10]:
                print(f"❌ {resposta_completa}")
                break

            # Regex para pegar a resposta final
            match = re.search(r'RESPOSTA FINAL:\s*([A-E])', resposta_completa, re.IGNORECASE)

            predicao = "?"
            if match:
                predicao = match.group(1).upper()
            else:
                # Fallback
                letras = re.findall(r'\b([A-E])\b', resposta_completa[-50:])
                if letras: predicao = letras[-1]

            print(f"⏱️ Tempo: {tempo:.1f}s")
            # Mostra o finalzinho para provar que ele fez a Verificação Final
            print(f"🧠 Trecho Final:\n...{resposta_completa[-400:].strip()}")
            print(f"\n🎯 PREDIÇÃO: {predicao}")

            if predicao == gabarito_oficial:
                print("✅ ACERTOU!")
                acertos += 1
            else:
                print("❌ ERROU.")

            total += 1

        except Exception as e:
            print(f"⚠️ Erro no loop: {e}")

    if total > 0:
        print(f"\n📊 PLACAR FINAL: {(acertos/total)*100:.1f}% de Acurácia")

# Roda o simulado
rodar_simulado_full(3)

🔥 INICIANDO SIMULADO COM METODOLOGIA COMPLETA (Sabiá-3.1)...

🔸 QUESTÃO 1 | Gabarito Oficial: [D]
🤔 Sabiá-3.1 aplicando os 7 passos (Pode demorar +20s)...
⏱️ Tempo: 33.9s
🧠 Trecho Final:
...*: Alternativa D é a mais alinhada com o objetivo de conservação e preservação.

### PASSO 7: VERIFICAÇÃO FINAL RIGOROSA
- ✅ Minha resposta calculada corresponde à alternativa D.
- ✅ Eliminei as alternativas A, B, C e E.
- ✅ Validei com interpretação cuidadosa do contexto.
- ✅ Verifiquei que a resposta faz sentido matematicamente e contextualmente.

Portanto, a resposta final é:

RESPOSTA FINAL: D

🎯 PREDIÇÃO: D
✅ ACERTOU!

🔸 QUESTÃO 2 | Gabarito Oficial: [A]
🤔 Sabiá-3.1 aplicando os 7 passos (Pode demorar +20s)...
⏱️ Tempo: 27.0s
🧠 Trecho Final:
...entes com o cenário.
- **Unidades e Contexto**: Corretos e coerentes.

**RESPOSTA FINAL**:
C) 426 milhões.

Note que a resposta foi baseada em suposições de dados do gráfico, pois os dados exatos não foram fornecidos. Se os dados reais do gráfico fossem 

In [24]:
!pip install -q -U torch torchvision torchaudio
!pip install -q -U transformers datasets trl peft bitsandbytes accelerate
print("✅ Ambiente de Treino Configurado.")

✅ Ambiente de Treino Configurado.


In [25]:
import json
import glob
import pandas as pd
from datasets import Dataset

def preparar_dataset_treino_robust():
    arquivos = glob.glob("enem_dados/*.jsonl")
    dados_processados = []

    print(f"🍳 Analisando {len(arquivos)} arquivos para treino...")

    questoes_encontradas = 0

    for arquivo in arquivos:
        with open(arquivo, 'r', encoding='utf-8') as f:
            for i, linha in enumerate(f):
                if not linha.strip(): continue

                try:
                    item = json.loads(linha)

                    # --- ESTRATÉGIA DE EXTRAÇÃO ROBUSTA ---
                    # 1. Tenta pegar o texto da questão (Várias tentativas de nome)
                    pergunta = (item.get('question') or
                                item.get('question_text') or
                                item.get('enunciado') or
                                item.get('text') or
                                item.get('body') or
                                item.get('texto'))

                    # 2. Tenta pegar o gabarito
                    gabarito = (item.get('correct_answer') or
                                item.get('gabarito') or
                                item.get('answer') or
                                item.get('label'))

                    # 3. Tenta pegar alternativas para enriquecer o contexto
                    alts = (item.get('alternatives') or
                            item.get('alternativas') or
                            item.get('answers') or [])

                    texto_alts = ""
                    if isinstance(alts, list):
                        letras = ['A', 'B', 'C', 'D', 'E']
                        for idx, txt in enumerate(alts):
                            if idx < 5: texto_alts += f"{letras[idx]}) {txt}\n"
                    elif isinstance(alts, dict):
                        for k, v in alts.items(): texto_alts += f"{k}) {v}\n"

                    # Se achou Pergunta e Gabarito, monta o treino
                    if pergunta and gabarito:
                        # Monta a pergunta completa com alternativas
                        pergunta_completa = f"{pergunta}\n\nAlternativas:\n{texto_alts}"

                        # Formato Instrucional (Alpaca style)
                        texto_treino = f"""### Instrução:
Você é um especialista no ENEM. Resolva a questão abaixo explicando o raciocínio.

### Questão:
{pergunta_completa}

### Resposta:
A alternativa correta é {gabarito}.
"""
                        dados_processados.append({"text": texto_treino})
                        questoes_encontradas += 1

                    # DIAGNÓSTICO DE ERRO (Só imprime na primeira linha do primeiro arquivo se falhar)
                    elif questoes_encontradas == 0 and i == 0:
                        print(f"⚠️ Aviso: Na primeira linha, não achei 'pergunta' ou 'gabarito'.")
                        print(f"   Chaves encontradas no JSON: {list(item.keys())}")

                except Exception as e:
                    print(f"Erro ao ler linha {i}: {e}")
                    continue

    print(f"✅ Processamento concluído: {questoes_encontradas} questões válidas encontradas.")

    if questoes_encontradas == 0:
        print("❌ ERRO CRÍTICO: O dataset continua vazio. Verifique os prints acima.")
        return None

    # Conversão para Dataset HuggingFace
    dataset = Dataset.from_pandas(pd.DataFrame(dados_processados))

    # Split Treino/Teste
    dataset = dataset.train_test_split(test_size=0.05)

    print(f"📦 Dataset final pronto para Fine-Tuning!")
    print(f"   Treino: {len(dataset['train'])} | Teste: {len(dataset['test'])}")

    return dataset

# Executa
dataset_enem = preparar_dataset_treino_robust()

# Validação visual
if dataset_enem:
    print("\n🔎 EXEMPLO DE DADO DE TREINO:")
    print("-" * 40)
    print(dataset_enem['train'][0]['text'][:500] + "...")
    print("-" * 40)

🍳 Analisando 21 arquivos para treino...
✅ Processamento concluído: 3084 questões válidas encontradas.
📦 Dataset final pronto para Fine-Tuning!
   Treino: 2929 | Teste: 155

🔎 EXEMPLO DE DADO DE TREINO:
----------------------------------------
### Instrução:
Você é um especialista no ENEM. Resolva a questão abaixo explicando o raciocínio.

### Questão:
Nesse sentido, cada autor caracterizou a atitude dos sertanejos, respectivamente, como fruto da

Alternativas:
A) manipulação e incompetência.
B) ignorância e solidariedade.
C) hesitação e obstinação.
D) esperança e valentia.
E) bravura e loucura.


### Resposta:
A alternativa correta é E.
...
----------------------------------------


In [28]:
import torch
import gc
import inspect
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
import os

# --- 1. LIMPEZA DE MEMÓRIA (Pra não dar OOM na T4) ---
gc.collect()
torch.cuda.empty_cache()

MODEL_ID = "/content/drive/MyDrive/modelos/sabia-7b"
OUTPUT_DIR = "/content/drive/MyDrive/modelos/sabia-7b-enem-finetuned"

# --- 2. CARREGAR MODELO (4-BIT) ---
print("🔄 Carregando Sabiá-7B para Treino...")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map={"": 0},
    use_cache=False, # Importante: Desliga cache para treino
    attn_implementation="sdpa"
)
model = prepare_model_for_kbit_training(model)

MAX_SEQUENCE_LENGTH = 512 # Definido aqui para uso no tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, local_files_only=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.model_max_length = MAX_SEQUENCE_LENGTH # Configura o tokenizer para truncar

# --- 3. CONFIGURAÇÃO LORA (Onde ele aprende) ---
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=32,            # Rank 32: Bom equilíbrio inteligência/memória
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"] # Treina todas as camadas de atenção
)

# --- 4. ARGUMENTOS DE TREINO (Compatibilidade Máxima) ---
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=1,              # 1 Passada completa (3k questões)
    per_device_train_batch_size=2,   # Batch pequeno para T4
    gradient_accumulation_steps=4,   # Compensa o batch pequeno
    gradient_checkpointing=True,     # Salva MUITA memória
    optim="paged_adamw_32bit",
    logging_steps=25,                # Log a cada 25 passos
    save_steps=100,                  # Salva checkpoint a cada 100
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="none"
)

# --- 5. DETECÇÃO DE VERSÃO (O Pulo do Gato) ---
# Isso impede o erro de 'tokenizer' vs 'processing_class'
trainer_kwargs = {
    "model": model,
    "train_dataset": dataset_enem["train"],
    "eval_dataset": dataset_enem["test"],
    "peft_config": peft_config,
    "args": training_args
    # "max_seq_length": MAX_SEQUENCE_LENGTH, # Removido para evitar TypeError
    # "packing": False # Removido para evitar TypeError
}

# Verifica dinamicamente o que a biblioteca instalada quer
sig = inspect.signature(SFTTrainer.__init__)
if "processing_class" in sig.parameters:
    print("   -> Versão Nova do TRL detectada.")
    trainer_kwargs["processing_class"] = tokenizer
else:
    print("   -> Versão Clássica do TRL detectada.")
    trainer_kwargs["tokenizer"] = tokenizer

# Função de formatação simples
def formatar_texto(exemplo):
    return exemplo['text']
trainer_kwargs["formatting_func"] = formatar_texto

# --- 6. INICIAR TREINO ---
print(f"🏋️‍♂️ Iniciando Fine-Tuning em {len(dataset_enem['train'])} questões...")
trainer = SFTTrainer(**trainer_kwargs)

trainer.train()

# --- 7. SALVAR ---
print("💾 Salvando modelo tunado...")
trainer.model.save_pretrained(OUTPUT_DIR)
print(f"🏆 SUCESSO! Adaptador salvo em: {OUTPUT_DIR}")

🔄 Carregando Sabiá-7B para Treino...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

   -> Versão Nova do TRL detectada.
🏋️‍♂️ Iniciando Fine-Tuning em 2929 questões...


Applying formatting function to train dataset:   0%|          | 0/2929 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/2929 [00:00<?, ? examples/s]

...

Truncating eval dataset:   0%|          | 0/155 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,1.124600
50,0.675000
75,1.052300
100,0.654100
125,0.959500
150,0.669000
175,0.990400
200,0.654200
225,0.950900
250,0.673200


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/pyt

💾 Salvando modelo tunado...
🏆 SUCESSO! Adaptador salvo em: /content/drive/MyDrive/modelos/sabia-7b-enem-finetuned


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# --- 1. CARREGAMENTO (Se já estiver carregado, ele pula) ---
if 'model' not in globals() or 'tokenizer' not in globals():
    print("🔄 Recarregando Sabiá-7B...")
    BASE_MODEL = "/content/drive/MyDrive/modelos/sabia-7b"
    ADAPTER_PATH = "/content/drive/MyDrive/modelos/sabia-7b-enem-finetuned"

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL, quantization_config=bnb_config, device_map={"": 0}
    )
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, local_files_only=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
    model.eval()

# --- 2. PROMPT COMPACTO (Focando no essencial para não estourar memória) ---
def resolver_enem_compacto(enunciado, alternativas_texto):
    questao_completa = f"{enunciado}\n\nAlternativas:\n{alternativas_texto}"

    # Prompt Otimizado: Instrução Curta + Gatilho de Raciocínio
    prompt_final = f"""### Instrução:
Você é um especialista no ENEM. Analise a questão, elimine as alternativas erradas e justifique a correta.

### Questão:
{questao_completa}

### Resposta (Análise e Gabarito):
1. Análise:"""

    # Tokenização com TRUNCAMENTO (Segurança contra estouro)
    inputs = tokenizer(prompt_final, return_tensors="pt", truncation=True, max_length=1500).to("cuda")

    print("🤖 Sabiá Local Processando...")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=400,      # Suficiente para explicar sem divagar
            do_sample=True,
            temperature=0.2,         # Baixa temperatura para manter o foco
            top_p=0.90,
            repetition_penalty=1.2,  # AUMENTEI para matar o loop "enempenho"
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    texto_full = tokenizer.decode(outputs[0], skip_special_tokens=False)

    # Limpeza: Pega o que vem depois do prompt e corta o fim
    resposta = texto_full.split("### Resposta (Análise e Gabarito):")[-1]
    resposta = resposta.split("</s>")[0] # Corta o token de fim

    print("="*60)
    print(f"❓ QUESTÃO RESUMIDA: {enunciado[:60]}...")
    print("-" * 60)
    print(f"🤖 SABIÁ RESPONDENDO:\n1. Análise:{resposta.strip()}")
    print("="*60)

# --- 3. TESTE FINAL ---

q1 = "A mitocôndria é uma organela celular relacionada à respiração celular aeróbica. É nela que ocorre o ciclo de Krebs e a cadeia respiratória. Considerando sua função, espera-se encontrar um grande número de mitocôndrias em células que:"
alts1 = "A) Realizam pouca atividade metabólica.\nB) Necessitam de grande fornecimento de energia (ATP).\nC) São procariontes e anaeróbicas.\nD) Estão mortas ou em decomposição.\nE) Realizam fotossíntese."

q2 = "Na frase 'O autor criticou a corrupção', o termo 'a corrupção' exerce qual função sintática?"
alts2 = "A) Sujeito.\nB) Objeto Direto.\nC) Objeto Indireto.\nD) Predicativo do Sujeito.\nE) Adjunto Adnominal."

resolver_enem_compacto(q1, alts1)
resolver_enem_compacto(q2, alts2)

🤖 Sabiá Local Processando...
❓ QUESTÃO RESUMIDA: A mitocôndria é uma organela celular relacionada à respiraçã...
------------------------------------------------------------
🤖 SABIÁ RESPONDENDO:
1. Análise:1. Análise: A questão aborda o tema mitocôndria, organela celular relacionada com o processo de respiração celular aeróbica.
2. A alternativa correta é B.
3. Justificativa: A alternativa correta é B, pois a mitocôndria é responsável pela produção de ATP, a partir da respiração celular aeróbica.
🤖 Sabiá Local Processando...
❓ QUESTÃO RESUMIDA: Na frase 'O autor criticou a corrupção', o termo 'a corrupçã...
------------------------------------------------------------
🤖 SABIÁ RESPONDENDO:
1. Análise:1. Análise: A alternativa correta é D. O predicativo do sujeito "corrupção" atua como adjunto adnominal do substantivo "crítica".
2. Gabarito: D.


In [8]:
import json
import glob
import pandas as pd
import re
from collections import Counter

# --- 1. TAXONOMIA ESPECIALISTA (LINGUAGENS REFINADA) ---
TAXONOMIA_ENEM = {
    "MATEMÁTICA": {
        "Matemática Básica": ["regra de três", "porcentagem", "razão", "proporção", "escala", "média", "mediana", "moda", "leitura de gráfico", "operações básicas", "financeira"],
        "Geometria Plana": ["área", "perímetro", "triângulo", "quadrado", "retângulo", "círculo", "polígono", "ângulo", "teorema de pitágoras"],
        "Geometria Espacial": ["volume", "prisma", "cilindro", "cone", "esfera", "cubo", "sólido", "capacidade", "projeção", "ortogonal"],
        "Geometria Analítica": ["plano cartesiano", "reta", "circunferência", "distância entre pontos", "equação da reta"],
        "Álgebra e Funções": ["função afim", "função quadrática", "exponencial", "logaritmo", "equação", "parábola", "juros compostos", "f(x)"],
        "Probabilidade/Combinatória": ["probabilidade", "análise combinatória", "fatorial", "agrupamento", "chance", "combinatória"]
    },
    "NATUREZA": {
        "Física - Mecânica": ["movimento", "velocidade", "força", "energia cinética", "potência", "leis de newton", "gravidade", "trabalho", "mecânica"],
        "Física - Eletricidade": ["circuito", "corrente", "resistor", "tensão", "potência elétrica", "campo elétrico", "magnético", "ohm"],
        "Física - Ondulatória/Óptica": ["onda", "frequência", "som", "luz", "refração", "espelho", "lente", "interferência", "difração"],
        "Física - Termologia": ["calor", "temperatura", "dilatação", "termodinâmica", "aquecimento", "térmico"],
        "Química Geral/Inorgânica": ["átomo", "tabela periódica", "ligação", "ácido", "base", "sal", "óxido", "estequiometria", "mol", "separação de misturas", "tabela"],
        "Química Orgânica": ["cadeia carbônica", "função orgânica", "álcool", "isomeria", "petróleo", "polímero", "biocombustível", "carbono"],
        "Química Físico-Química": ["pH", "pilha", "eletrólise", "termoquímica", "cinética", "equilíbrio químico", "solução", "concentração"],
        "Biologia - Ecologia": ["cadeia alimentar", "impacto ambiental", "poluição", "ciclo", "bioma", "relações ecológicas", "preservação", "sustentabilidade"],
        "Biologia - Citologia/Genética": ["célula", "dna", "rna", "membrana", "divisão celular", "biotecnologia", "vacina", "vírus", "genética"],
        "Biologia - Fisiologia": ["corpo humano", "sistema", "digestão", "respiração", "hormônio", "doença", "saúde", "fisiologia"]
    },
    "HUMANAS": {
        "História do Brasil": ["colônia", "escravidão", "império", "vargas", "ditadura", "militar", "populismo", "constituição", "independência", "brasil"],
        "História Geral": ["guerra fria", "revolução industrial", "segunda guerra", "fascismo", "nazismo", "idade média", "antiguidade", "imperialismo", "revolução"],
        "Geografia Física": ["clima", "relevo", "solo", "hidrografia", "vegetação", "impacto ambiental", "tectônica", "geologia"],
        "Geografia Humana/Geopolítica": ["globalização", "população", "urbanização", "migração", "agricultura", "indústria", "território", "economia", "geopolítica"],
        "Filosofia": ["ética", "política", "conhecimento", "ciência", "moral", "liberdade", "filósofo", "razão", "mito"],
        "Sociologia": ["cultura", "trabalho", "desigualdade", "movimentos sociais", "cidadania", "estado", "poder", "democracia", "sociedade"]
    },
    "LINGUAGENS": {
        "Interpretação de Textos (Português)": [
            "estratégia argumentativa", "recurso expressivo", "efeito de sentido", "tese", "ponto de vista",
            "coesão", "coerência", "inferência", "objetivo do texto", "gênero textual", "tipologia",
            "função da linguagem", "intertextualidade", "semântica", "vocabulário", "norma padrão",
            "variação linguística", "regionalismo", "leitura", "verbal", "não verbal"
        ],
        "Literatura Brasileira": ["modernismo", "romantismo", "barroco", "realismo", "machado de assis", "poema", "poesia", "lírico", "narrativa literária", "autor"],
        "Artes": ["vanguarda", "pintura", "estética", "visual", "arte contemporânea", "museu", "instalação", "performance", "artístico"],
        "Educação Física/Corpo": ["esporte", "saúde", "corpo", "exercício", "jogo", "dança", "sedentarismo", "lazer", "prática corporal"],
        "Língua Estrangeira (Inglês)": ["english", "cartoon", "advertisement", "lyrics", "poem in english", "language", "text"],
        "Língua Estrangeira (Espanhol)": ["español", "historieta", "viñeta", "lengua", "poema en español", "hablante", "texto"]
    }
}

def classificar_tema_v3(texto):
    texto_lower = texto.lower()
    scores = Counter()

    # Varredura inteligente
    for area, subareas in TAXONOMIA_ENEM.items():
        for subarea, keywords in subareas.items():
            for kw in keywords:
                if kw in texto_lower:
                    # Peso extra se for indício forte de língua estrangeira no contexto certo
                    if "Língua Estrangeira" in subarea and len(texto) < 1000:
                         scores[(area, subarea)] += 2
                    else:
                         scores[(area, subarea)] += 1

    if not scores:
        return "OUTROS", "Geral"

    # Retorna o campeão
    (area_vencedora, subarea_vencedora), _ = scores.most_common(1)[0]
    return area_vencedora, subarea_vencedora

def gerar_oraculo_v3():
    print("🔮 Iniciando Oráculo V3 (Foco em Linguagens Corrigido)...")
    arquivos = glob.glob("enem_dados/*.jsonl")

    dados = []

    for arquivo in arquivos:
        # Tenta extrair o ano do nome do arquivo
        ano_match = re.search(r'20\d{2}', arquivo)
        ano = int(ano_match.group(0)) if ano_match else 2020

        with open(arquivo, 'r', encoding='utf-8') as f:
            for linha in f:
                try:
                    item = json.loads(linha)
                    texto = (item.get('question') or item.get('enunciado') or item.get('text') or "")

                    if texto and len(texto) > 30:
                        area, subarea = classificar_tema_v3(texto)
                        if area != "OUTROS":
                            dados.append({"Ano": ano, "Area": area, "Subarea": subarea})
                except: pass

    df = pd.DataFrame(dados)
    if df.empty: return print("❌ Sem dados classificados.")

    # Peso Temporal (Anos recentes valem mais)
    df['Peso'] = 1 + (df['Ano'] - df['Ano'].min()) * 0.4

    print("\n" + "█"*60)
    print("📊 RAIO-X ESTRATÉGICO DO ENEM (PREDIÇÃO)")
    print("█"*60)

    # LOOP CORRIGIDO AQUI:
    for area in TAXONOMIA_ENEM.keys():
        print(f"\n📘 ÁREA: {area}")
        print("-" * 50)

        df_area = df[df['Area'] == area]
        if df_area.empty:
            print("   (Dados insuficientes)")
            continue

        ranking = df_area.groupby('Subarea')['Peso'].sum().sort_values(ascending=False)
        total = ranking.sum()

        for sub, score in ranking.items():
            prob = (score / total) * 100
            blocos = int(prob / 2)
            barra = "█" * blocos + "░" * (25 - blocos)
            print(f"{sub.ljust(35)} | {prob:.1f}% {barra}")

# Executar
gerar_oraculo_v3()

🔮 Iniciando Oráculo V3 (Foco em Linguagens Corrigido)...

████████████████████████████████████████████████████████████
📊 RAIO-X ESTRATÉGICO DO ENEM (PREDIÇÃO)
████████████████████████████████████████████████████████████

📘 ÁREA: MATEMÁTICA
--------------------------------------------------
Matemática Básica                   | 28.8% ██████████████░░░░░░░░░░░
Geometria Espacial                  | 27.4% █████████████░░░░░░░░░░░░
Geometria Plana                     | 22.2% ███████████░░░░░░░░░░░░░░
Geometria Analítica                 | 10.3% █████░░░░░░░░░░░░░░░░░░░░
Probabilidade/Combinatória          | 8.6% ████░░░░░░░░░░░░░░░░░░░░░
Álgebra e Funções                   | 2.7% █░░░░░░░░░░░░░░░░░░░░░░░░

📘 ÁREA: NATUREZA
--------------------------------------------------
Química Geral/Inorgânica            | 26.0% ████████████░░░░░░░░░░░░░
Física - Mecânica                   | 17.8% ████████░░░░░░░░░░░░░░░░░
Biologia - Citologia/Genética       | 12.4% ██████░░░░░░░░░░░░░░░░░░░
Biologia - F